In [5]:
import pandas as pd
import numpy as np
import xarray as xr
import pygrib
import datetime
import numexpr as ne

In [4]:
# ds = xr.open_dataset("gfsdowntemp/20231117t00.f001", engine="cfgrib") <<<------ GABISA

In [6]:
grbs = pygrib.open('gfsdowntemp/20231117t00.f001')

In [ ]:
descr=[]
for grb in grbs:
    descr.append(str(grb).split(':'))
df = pd.DataFrame(
    descr, 
    columns=['id', 'name', 'unit', 'spacing', 'layer', 'level', 'hour', 'run_dt'])
df.name.unique()

In [ ]:
df.loc[df.name == 'Temperature'].values

In [63]:
grbs.seek(0)
grb1 = grbs.read(100)[0:100] # read returns a list with the next N (N=1 in this case) messages.
grb1

[1:Visibility:m (instant):regular_ll:surface:level 0:fcst time 1 hrs:from 202311170000,
 2:Wind speed (gust):m s**-1 (instant):regular_ll:surface:level 0:fcst time 1 hrs:from 202311170000,
 3:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 10000 Pa:fcst time 1 hrs:from 202311170000,
 4:Temperature:K (instant):regular_ll:isobaricInhPa:level 10000 Pa:fcst time 1 hrs:from 202311170000,
 5:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 10000 Pa:fcst time 1 hrs:from 202311170000,
 6:Total Cloud Cover:% (instant):regular_ll:isobaricInhPa:level 10000 Pa:fcst time 1 hrs:from 202311170000,
 7:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 10000 Pa:fcst time 1 hrs:from 202311170000,
 8:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 10000 Pa:fcst time 1 hrs:from 202311170000,
 9:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 15000 Pa:fcst time 1 hrs:from 202311170000,
 10:Temperature:K (instant):regular_l

In [5]:
def mstoknot(var):
    return ne.evaluate('var*1.9438445')

def KtodegC(var):
    return ne.evaluate('var-273.15')

def patombar(var):
    return ne.evaluate('var*0.01')

In [8]:
g = grbs[1]
dtni = g.analDate + datetime.timedelta(hours = g['endStep'])
shortnames = []
for g in grbs:
    if g.level not in shortnames:
        shortnames.append(g.level)

print(shortnames)

[]


In [9]:
levels = []
gselz = grbs.select(shortName = 'u')
for lev in gselz:
     levels.append(float(lev.level))

In [10]:
levels

[100.0,
 150.0,
 200.0,
 250.0,
 300.0,
 400.0,
 500.0,
 600.0,
 700.0,
 800.0,
 850.0,
 900.0,
 925.0,
 950.0,
 1000.0]

In [6]:
param2d = [['tp','rr',None,'mm'],
           ['lftx','li',None,'index'],
           ['cape','cape',None,'J/kg'],
#            ['prmsl','mslp',patombar,'mbar'],
           ['tcc','tcc',None,'percent'],
           
           ['10u','u10',None,'m s**-1'],
           ['10v','v10',None,'m s**-1'],
#            ['2d','dp2',KtodegC,'degC'],
           ['2r','rh2',None,'%'],
           ['2t','t2',KtodegC,'degC'],
           
           ]

param2d = [
    ['2r', 'rh2', None, '%'],                  # Relative humidity at 2m
    ['2t', 't2', KtodegC, 'degC'],             # Temperature at 2m
    ['10u', 'u10', None, 'm s**-1'],           # U-component of wind at 10m
    ['10v', 'v10', None, 'm s**-1'],           # V-component of wind at 10m
    ['vis', 'vis', None, 'm'],                 # Visibility
    ['gust', 'gust', None, 'm s**-1'],         # Wind gust
    ['prate', 'prate', None, 'kg m**-2 s**-1'],# Precipitation rate
    ['tp', 'rr', None, 'mm'],                  # Total precipitation
#     ['acpcp', 'acpcp', None, 'kg m**-2'],      # Accumulated convective precipitation
    ['lftx', 'lftx', None, 'K'],               # Surface lifted index
    ['cape', 'cape', None, 'J kg**-1'],        # Convective Available Potential Energy
]

param3d = [['u','u',None,'m s**-1'],
           ['v','v',None,'m s**-1'],
#            ['w','w',None,'Pa/s'],
           ['r','rh',None,'percent'],
           ['t','t',KtodegC,'degC'],
#            ['absv','absv',None,'S**-1'],
           ]

param3d = [
    ['u', 'u', None, 'm s**-1'],      # U-component of wind
    ['v', 'v', None, 'm s**-1'],      # V-component of wind
#     ['gh', 'gh', None, 'gpm'],        # Geopotential Height
    ['t', 't', None, 'K'],            # Temperature
    ['r', 'r', None, '%'],            # Relative Humidity
    ['tcc', 'tcc', None, '%'],        # Total Cloud Cover
]

# yang ada datanya tapi ga masuk sini orog

In [78]:
gsel = grbs.select(shortName = param2d[0][0])
g = gsel[0]

96:2 metre relative humidity:% (instant):regular_ll:heightAboveGround:level 2 m:fcst time 1 hrs:from 202311170000

In [55]:
lat, lon = g.latlons()
lat = lat[:,0]
lon = lon[0,:]

In [54]:
dt = '20231117'
cycl = '00'
ncout = 'gfs_%s_%s00v2.nc' % (dt,cycl)
gfspath = 'gfsdowntemp/nctemp/'
fo = netCDF4.Dataset( gfspath + ncout,'w')

In [56]:
fo.createDimension('time',None)
fo.createDimension('lat',lat.size)
fo.createDimension('lon',lon.size)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440

In [58]:
dtni = g.analDate + datetime.timedelta(hours = g['endStep'])

In [59]:
nc_lon = fo.createVariable('lon','d', ('lon'))
nc_lat = fo.createVariable('lat','d', ('lat'))
nc_t = fo.createVariable('time','d', ('time'))

In [61]:
fo.setncattr('ordinal',datetime.date.toordinal(dtni))

In [62]:
tunits = 'minutes since %s' % dtni.strftime('%Y-%m-%d %H:%M')
nc_t.units = tunits

nc_lon.units = 'degrees_east'
nc_lat.units = 'degrees_north'
nc_lon.long_name = 'Longitude'
nc_lat.long_name = 'Latitude'

nc_lon[:] = lon
nc_lat[:] = lat

In [70]:
lat.size

721

In [79]:
data = g['values']

In [89]:
ln = g['name']

(721, 1440)

In [ ]:
def fno_open(outnc, gsel, dtn, hasz = False,dom = 'global'):
    g = gsel[0]
    
    lat, lon = g.latlons()
    lat = lat[:,0]
    if dom == 'global':
        lat = np.flip(lat,0)
    lon = lon[0,:]
    
    fo = netCDF4.Dataset(outnc,'w')
    
    fo.createDimension('time',None)
    fo.createDimension('lat',lat.size)
    fo.createDimension('lon',lon.size)
    
    fo.setncattr('ordinal',datetime.date.toordinal(dtn))
    
    
    nc_t = fo.createVariable('time','d', ('time'))
    nc_lon = fo.createVariable('lon','d', ('lon'))
    nc_lat = fo.createVariable('lat','d', ('lat'))
    
    
    tunits = 'minutes since %s' % dtn.strftime('%Y-%m-%d %H:%M')
    nc_t.units = tunits
    
    nc_lon.units = 'degrees_east'
    nc_lat.units = 'degrees_north'
    nc_lon.long_name = 'Longitude'
    nc_lat.long_name = 'Latitude'
    
    nc_lon[:] = lon
    nc_lat[:] = lat
    
    level = None
    if hasz:
        level = []
        for i in range(len(gsel)):
            level.append(gsel[i].level)
        level = np.array(level)
        level[::-1].sort()
        fo.createDimension('level', len(gsel))
        nc_level = fo.createVariable('level','d', ('level'))
        nc_level.units = 'mbar'
        nc_level.long_name = 'isobaric level'
        nc_level[:] = level
        
    return fo, lon.size, lat.size, level

In [27]:
fo = None
ti = 0
if not fo:
    dtnr = dtni
    if len(param3d) != 0:
        gsel = grbs.select(shortName = param3d[0][0])
        hasz = True
        fo, nx, ny, level = fno_open(putnc, gsel, dtni, hasz = hasz, dom = dom)
    else:
        gsel = grbs.select(shortName = param2d[0][0])
        hasz = False
        fo, nx, ny, level = fno_open(putnc, gsel, dtni, hasz = hasz, dom = dom)

NameError: name 'fno_open' is not defined

In [19]:
def gendts():
    import datetime
    dtn = datetime.datetime.utcnow()
#    dtn = dtn = datetime.datetime(2018,9,18,17,59,0)
    if (dtn.hour >= 3) & (dtn.hour < 9):
        cycl = '00'
    elif(dtn.hour >= 9) & (dtn.hour <15):
        cycl = '6'
    elif (dtn.hour >= 15) & (dtn.hour < 21):
        cycl = '12'
    elif (dtn.hour >= 21) & (dtn.hour < 24):
        cycl = '12'
    else:
        cycl = '18'
        dtn = dtn - datetime.timedelta(hours = 12)
    
    dt = dtn.strftime('%Y%m%d')
    
    return dt,cycl

In [20]:
gendts()

('20231122', '00')